## Fuzzy String Matching
Fuzzy String Matching, also known as Approximate String Matching, is the process of finding strings that approximately match a pattern. 

In Python, Fuzzywuzzy is a library that uses Levenshtein Distance to calculate the differences between sequences and patterns. 

In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

### Data Loading and preprocessing

In [2]:
df_dbpedia = pd.read_csv("/Users/twei/workplace/ISE-workplace/ISE-Linking-Entities-from-Images-to-Knowledge-Graphs/src/data/dbpedia_classes.csv")
df_imageNet = pd.read_csv("/Users/twei/workplace/ISE-workplace/ISE-Linking-Entities-from-Images-to-Knowledge-Graphs/src/data/ImageNet.csv")
df_imageNet['Class Name'] = df_imageNet['Class Name'].str.replace(", ",",")
df_imageNet['Class Name'] = df_imageNet['Class Name'].str.split(",")

In [3]:
df_imageNet.iloc[1]['Class Name'] 

['goldfish', 'Carassius auratus']

### String Distance Calculating
As we can see, in each row we have a list of string. The initial prototyping idea is iterate each imageNet class list with dbpedia Class and chosing the highest sequences’ similarity ratio of the string item in the imageNet class list, because the strings are Synonyms with probabaly different letters. 

In [4]:
def get_max_fuzzy_ratio():
    max_ratio_list = []
    max_label_list = []
    for class_list in df_imageNet['Class Name']:
        max_val = 0
        max_label = ''
        for item in class_list:
            for label in df_dbpedia['label']:
                if max_val< fuzz.ratio(item, label):
                    max_val = fuzz.ratio(item, label)
                    max_label = label
        max_ratio_list.append(max_val)
        max_label_list.append(max_label)
        
    df_imageNet['max_similarity_ratio'] = max_ratio_list
    df_imageNet['dbpedia_class'] = max_label_list
    df_imageNet.rename(columns = {"Class Name":"imageNet_class"}, inplace=True)
            
get_max_fuzzy_ratio()


In [5]:
df_imageNet

,Class ID,imageNet_class,max_similarity_ratio,dbpedia_class
0,0,"[tench, Tinca tinca]",60,beach
1,1,"[goldfish, Carassius auratus]",67,fish
2,2,"[great white shark, white shark, man-eater, ma...",67,monastery
3,3,"[tiger shark, Galeocerdo cuvieri]",58,figure skater
4,4,"[hammerhead, hammerhead shark]",62,camera
...,...,...,...,...
995,995,[earthstar],67,artist
996,996,"[hen-of-the-woods, hen of the woods, Polyporus...",50,unit of work
997,997,[bolete],67,letter
998,998,"[ear, spike, capitulum]",86,year


In [9]:
df_imageNet.sort_values(ascending = False, by= "max_similarity_ratio")[:30]

,Class ID,imageNet_class,max_similarity_ratio,dbpedia_class
487,487,"[cellular telephone, cellular phone, cellphone...",100,mobile phone
743,743,"[prison, prison house]",100,prison
525,525,"[dam, dike, dyke]",100,dam
323,323,"[monarch, monarch butterfly, milkweed butterfl...",100,monarch
981,981,"[ballplayer, baseball player]",100,baseball player
980,980,[volcano],100,volcano
663,663,[monastery],100,monastery
979,979,"[valley, vale]",100,valley
916,916,"[web site, website, internet site, site]",100,website
668,668,[mosque],100,mosque


### Evaluation
Without ground-truth we could not do quantatitive evaluation properly. But with an overview, we could see that 
- String Matching recognizes the class Mapping that based on a alphabetic similarity. For example, "[electric ray, crampfish, numbfish, torpedo]"	 is mapped to the class fish
- But for imageNet classes, that are semantic similar to the dbpedia class, string mapping do not work well. For example, hen is mapped to chef. To solve this problem, we may need a NLP model to do high level mapping.